本notebook：
- 修改prompt，让llm将职责拆成两部分
- 根据岗位名称筛选特定大领域岗位信息
- 取llm返回的每个岗位信息的embedding，求余弦相似度，使相似性过强的岗位信息共用一个提取结果

In [94]:
import pandas as pd

data = pd.read_csv("D:\AI\AI-job-analysis\data_merge\merged_data_1222新.csv")
cv_condition = data["岗位名称"].str.contains("CV|计算机视觉|图像")

In [95]:
cv_data = data[cv_condition]
# 小批量测试数据
mini_cv_data = cv_data[:10]
len(cv_data)

94

In [96]:
mini_cv_data.iloc[0]["岗位要求"]

'岗位需求介绍：分为三大方向\n第一，机器学习，包括NLP、CV算法，做预训练方向；\n第二，强化学习：RLHF方向；\n第三，AI infra：大规模分布式训练和推理框架方向\n社招岗位寻访方向：\n1】硬性要求：年轻、高潜，国内、外名校本、硕、博士，学历越高越好，获得计算机、数学等相关奖项越多越好，工作2~5年+，年纪不能太大，对标阿里P6及以上，字节2-1及以上。\n第一梯队人选：满足第一条，且是阿里校招生--阿里星、百度校招生-百度AI DU、快手校招生-快Star、字节校招生2-1及以上级别的。即曾具备这些校招标签的人选~\n第二梯队人选：满足第一条，且在知名互联网公司，正在从事大模型相关算法、机器学习等研发的年轻、高潜同学，对标职级能到阿里P6、字节2-1及以上的，\n数据质量模型，画像：满足其一即可\n1、搜索、评论、资讯类等场景长文本内容理解/类目/标签模型\n2、基座模型预训练数据处理\n预训练：语言模型，画像：满足其一即可\n1、优先看基座模型预训练\n2、或者垂类模型做Continue train的也可以，认知要好，nlp的深度要好。\n3、NLP场景能力够强也是ok的，核心还是看认知，\n预训练-多模态，画像：\n1、做过多模态大模型的理解（关键词表征学习）或者生成；经验导向，看有相关经验的'

In [97]:
import pickle

# 读取缓存文件，若缓存文件不存在或者为空（初次运行），则初始化一个空的字典
try:
    with open("new_dic.cache", "rb") as f:
        new_cache = pickle.load(f)
except:
    print("Empty cache file")
    new_cache = {}

try:
    with open("s2e.cache", "rb") as f:
        s2e = pickle.load(f)
except:
    print("Empty s2e file")
    s2e = {}

In [98]:
from test import *

# from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json


for text in cv_data["岗位要求"]:
    prompt = f"""
请总结招聘信息中岗位所涉及的所有领域类别、任务类别，提供的招聘信息在三对反引号中，返回为一个json字典。
步骤：
1. 请阅读三对反引号中的招聘信息，提取其中与岗位职责相关的内容
2. 将职责拆成领域和任务两部分，任务为该职责的实际工作内容，但不包含具体的技术领域方向。领域为技术方向，而非实际工作内容。例如“研究高精度的目标检测算法”中，领域为“目标检测”，任务为“算法研究”。
3. 任务应简练、概括性强，不得出现定语、状语和形容词。职责中的定语状语形容词，可作为领域。
4. 将任务进行拆分，使得一个字符串中只描述一种任务，例如“研究和开发高精度的目标检测算法”，应当拆分为“算法研究”和“算法开发”。任务中不得出现顿号、/等分隔符，如果出现，应当拆分为多个任务。
5. 任务不包括沟通能力、领导力等素养类要求，只包括技术类要求。例如“具备良好的沟通能力和团队合作精神”，不应当出现在任务中。
6. 再次阅读招聘信息，将其中所提及的其他领域提取出来，包括广义的行业领域和岗位所需的某些领域经验等。例如“有目标跟踪、自动驾驶等经验”，则领域中除前几步提取出的职责领域外，还应当包含“目标跟踪”、“自动驾驶”。
7. 将处理好的领域和任务填入对应的字典，返回的字典key为“领域”、“任务”，字典中必须包含且只能包含这两个键值对。
8. 若有多个领域或任务，对应键的值以列表形式返回，列表中的元素为字符串。
9. 若没有提及领域或任务，则该键的值为空列表。

```
{text}
```
"""
    if text not in s2e:
        # get_embedding后，将文本与之前的文本进行相似度比较，如果相似度大于0.9，则将之前的文本的处理结果复制给当前文本
        s2e[text] = get_embedding(text, model="text-embedding-ada-002")
        for k, v in s2e.items():
            if k != text:
                if cosine_similarity(np.array(v), np.array(s2e[text])) > 0.99:
                    print(k, "\n=========\n", text)
                    new_cache[text] = new_cache[k]
                    break
    # 如果文本已在缓存区，则跳过
    if text in new_cache:
        continue
    # 否则，进行ChatGPT_request
    jsons = ChatGPT_request(prompt)
    # print(text)
    print("====================================")
    print(jsons)
    # 将ChatGPT_request的结果存入缓存区
    new_cache[text] = json.loads(jsons)

{
    "领域": ["图像处理", "机器学习优化算法", "模型部署", "深度学习框架", "C/C++", "Linux平台下的开发"],
    "任务": ["算法优化迭代", "AI/CV技术跟进", "推动算法落地", "项目落地经验", "模型部署经验"]
}
{
  "领域": ["人脸识别", "视觉算法", "大模型", "图像加密", "图像处理"],
  "任务": ["模型研发", "模型压缩", "模型剪枝", "技术跟踪", "产线开发", "产线验证", "产线优化", "产线维护", "阅读论文", "深度学习建模"]
}
职位介绍
深度学习
图像检测
人脸识别
算法研究
深度学习框架
工作职责
1. 负责人脸身份认证领域算法模型研发，包括但不限于：人脸活体检测、人脸识别、人脸质量评估、人脸关键点检测、 图像加密、图像处理等方向的深度学习模型实际研发经验，具有炫彩活检研发经验者优先。 2. 负责深度学习模型压缩、模型剪枝等研发工作，具有便携终端轻量级模型研发经验者优先。 3. 负责跟踪上述领域的先进技术及研发工作。 4. 负责相关算法模型的产线开发、验证、优化及维护等产线工作。
任职要求
1. 计算机相关专业，大学本科毕业，硕士及以上学历者优先。 2. 具有良好的代码能力，精通各类深度学习框架下，模型研发，熟练掌握Python、Java、C++等至少一种语言。 3. 具备快速阅读深度学习相关论文并理解算法的能力。 4. 针对上述人脸相关方向的深度学习建模具有3年及以上经验。
其他信息
语言要求：不限
行业要求：互联网 
 职位介绍
深度学习
图像检测
人脸识别
算法研究
深度学习框架
工作职责
1.负责人脸身份认证领域算法模型研发，包括但不限于：人脸活体检测、人脸识别、人脸质量评估、人脸关键点检测、 图像加密、图像处理等方向的深度学习模型实际研发经验，具有炫彩活检研发经验者优先。
2. 负责深度学习模型压缩、模型剪枝等研发工作，具有便携终端轻量级模型研发经验者优先。
3.负责跟踪上述领域的先进技术及研发工作。
4. 负责相关算法模型的产线开发、验证、优化及维护等产线工作。
任职要求
1.计算机相关专业，大学本科毕业，硕士及以上学历者优先。
2.具有良好的代码能力，精通各类深度学习框架下，模型研发，熟练掌握Python、Jav

In [99]:
# import json
# new_cache = {k: json.loads(v) for k, v in new_cache.items()}

In [100]:
klong = []
for k, v in new_cache.items():
    if v["任务"]:
        if len(v["任务"][0]) >= 10:
            print(v)
            klong.append(k)

{'领域': ['深度学习', '算法研究', '图像处理', '计算机视觉'], '任务': ['研究和开发深度学习算法', '撰写技术描述文档', '撰写专利发明点', '发表影像处理论文']}
{'领域': ['医药专业', '市场推广', '产品策划', '团队管理'], '任务': ['制定部门产品中长期策略和发展方向', '策划推广活动和学术品牌建设项目', '设计推广物料和活动文案', '客情项目/活动的设计和推动', '优化KOL专家库', '制定客户管理办法', '完善数据管理体系', '创新处方药推广模式与推广方式', '团队的管理与培训指导']}
{'领域': ['计算机视觉', '多模态技术', '感知/识别', '深度学习', '网络架构', '机器学习', '模型压缩优化', '3D目标检测', 'BEV感知'], '任务': ['自动化生产和实时更新地图POI数据', '研究和开发2D/3D目标检测、分割、匹配、深度估计、位姿估计', '参与其他创新应用预研', '具备数学和计算机科学基础', '深入研究图像分类', '熟悉MLP、CNN、Transformer等网络架构', '具有编程实现能力', '快速学习能力']}
{'领域': ['计算机视觉', '多模态技术', '感知/识别', '深度学习框架', '机器学习', '模型压缩优化'], '任务': ['地图POI数据的自动化生产和实时更新', '2D/3D目标检测', '目标分割', '目标跟踪', 'BEV感知', '自监督学习', '网络架构研究和开发', '编程实现', '文献资料解决问题', '模型压缩优化']}
{'领域': ['计算机视觉', '多模态技术', '感知/识别', '深度学习', '网络架构', '机器学习', '模型压缩优化', '3D目标检测', 'BEV感知'], '任务': ['自动化生产和实时更新地图POI数据', '研究和开发2D/3D目标检测、分割、匹配、深度估计、位姿估计', '参与其他创新应用预研', '具备数学和计算机科学基础', '深入研究图像分类', '熟悉MLP、CNN、Transformer等网络架构', '具有编程实现能力', '快速学习能力']}
{'领域': ['计算机视觉', '多媒体', '机器学习', '深度学

In [101]:
with open("new_dic.cache", "wb") as f:
    pickle.dump(new_cache, f)

In [102]:
with open("s2e.cache", "wb") as f:
    pickle.dump(s2e, f)

In [103]:
len(new_cache)

402

In [104]:
len(s2e)

402